In [1]:
import json 
import numpy as np
import pandas as pd
from nltk_utils import tokenize, stem, bag_of_words

import tensorflow as tf

In [2]:
ignore_words = ["?", "i", "it", "a", "!", "me", "can", "you", "'s", "does", "of", "do", "my"]

# Train

In [3]:
# get tags and set of all words from training data
# xy contains sample prompt and its expected output category

with open("intents.json", "r") as f:
    intents = json.load(f)

tags = []
all_words = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    
    for pattern in intent["patterns"]:
        
        words = tokenize(pattern)
        l1 = [stem(w) for w in words if w.lower() not in ignore_words]
        
        all_words.extend(l1)
        xy.append((l1, tag))

all_words = sorted(set(all_words))
np.random.shuffle(xy)

In [4]:
print(tags)
print(all_words)
print(xy)

['greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
['accept', 'appreci', 'are', 'around', 'bye', 'card', 'cash', 'credit', 'deliv', 'deliveri', 'funni', 'get', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'name', 'onli', 'pay', 'paypal', 'realli', 'see', 'sell', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'time', 'up', 'what', 'when', 'which', 'with', 'ya', 'your']
[(['goodby'], 'goodbye'), (['which', 'item', 'have'], 'items'), (['realli', 'appreci'], 'thanks'), (['tell', 'someth', 'funni'], 'funny'), (['see', 'ya'], 'goodbye'), (['bye'], 'goodbye'), (['thank'], 'thanks'), (['deliv'], 'delivery'), (['hi'], 'greeting'), (['how', 'long', 'deliveri', 'take'], 'delivery'), (['thank'], 'thanks'), (['tell', 'joke'], 'funny'), (['are', 'cash', 'onli'], 'payments'), (['what', 'sell'], 'items'), (['take', 'credit', 'card'], 'payments'), (['hey'], 'greeting'), (['name', 'is'], '

In [5]:
# Prepare training set

X_train = []
y_train = []

for sentence, tag in xy:
    bag = bag_of_words(sentence, all_words)
    X_train.append(bag)

    label = bag_of_words(tag, tags)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [6]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

# Model

In [7]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=16, input_dim = len(all_words), activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

ann.add(tf.keras.layers.Dense(units=len(tags) , activation='softmax'))

ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) 

In [8]:
ann.fit(X_train, y_train, batch_size=5, epochs = 100)

Epoch 1/100


6/6 [==============================] - 0s 2ms/step - loss: 1.9235 - accuracy: 0.1667
Epoch 2/100
6/6 [==============================] - 0s 1ms/step - loss: 1.9031 - accuracy: 0.2000
Epoch 3/100
6/6 [==============================] - 0s 1ms/step - loss: 1.8868 - accuracy: 0.2000
Epoch 4/100
6/6 [==============================] - 0s 1000us/step - loss: 1.8695 - accuracy: 0.3333
Epoch 5/100
6/6 [==============================] - 0s 1ms/step - loss: 1.8544 - accuracy: 0.4000
Epoch 6/100
6/6 [==============================] - 0s 1ms/step - loss: 1.8389 - accuracy: 0.4333
Epoch 7/100
6/6 [==============================] - 0s 1ms/step - loss: 1.8240 - accuracy: 0.4333
Epoch 8/100
6/6 [==============================] - 0s 902us/step - loss: 1.8095 - accuracy: 0.4333
Epoch 9/100
6/6 [==============================] - 0s 1ms/step - loss: 1.7939 - accuracy: 0.4667
Epoch 10/100
6/6 [==============================] - 0s 1ms/step - loss: 1.7786 - accuracy: 0.4667
Epoch 11/100
6/6 [====

In [9]:
# Tag index of input sentence

input_1 = "Hello"

np.argmax(ann.predict(bag_of_words(input_1, all_words).reshape(1, -1)))

1/1 [==============================] - 0s 58ms/step


0

# Response

In [10]:
# Respond based on given set of training responses

def respond(sentence):

    '''
    input : 
        sentence : input prompt from user

    output : 
        response : text from given set of responses in training data
        tag :      tag of input prompt
    '''

    words = tokenize(sentence)
    sentence = [stem(w) for w in words if w not in ignore_words]
    bag = bag_of_words(sentence, all_words).reshape(1, -1)
    
    probs = ann.predict(bag)
    max_idx = np.argmax(probs)
    response = np.random.choice(intents['intents'][max_idx]['responses'])
    tag = intents['intents'][max_idx]['tag']

    return response, tag


In [11]:
input_2 = "what is your delivery time"

respond(input_2)

1/1 [==============================] - 0s 16ms/step


('Shipping takes 2-4 days', 'delivery')

# Speech

In [12]:
import speech_recognition as sr 
import pyttsx3

engine = pyttsx3.init()

In [13]:
# list of languages available in your windows

for voice in engine.getProperty('voices'):
    print(voice.id)

HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\MSTTS_V110_hiIN_KalpanaM


In [14]:
# Assigning language and voice

lang = 'english'
voc = engine.getProperty('voices')[1].id

engine.setProperty('voice', voc)

In [15]:
# Speech to text

def get_transcript(lang):

    ''' 
    input : 
        lang : language of voice input
        
    output : transcript, bool
        if no error, returns transcript of given voice input and true.
        else, returns error message and false.
    '''

    recognizer = sr.Recognizer()
    audio = ' '
    transcript = ' '

    print("\nListening ......\n")
    
    with sr.Microphone() as source:
        audio = recognizer.listen(source)

    try:
        transcript = recognizer.recognize_google(audio, language=lang)
        print(transcript)
        return  transcript, True
    
    except sr.UnknownValueError:
        print("Could not understand audio")
        return "Could not understand audio", False
    
    except sr.RequestError as e:
        print("Could not request results")
        return "Could not request results", False
 

In [16]:
# Text to speech

def speak(response):
    
    '''
    input : response 
        text to be converted to speech
    '''

    engine.say(response)
    engine.runAndWait()


# Main

In [17]:
while True :

    text, correct = get_transcript(lang)
    print("You : ", text)

    response, tag = respond(text)

    if correct :

        print("Chatbot : ", response)
        speak(response)

        if tag == 'goodbye' :
            break
    else :

        speak(text)


Listening ......

hello
You :  hello
1/1 [==============================] - 0s 16ms/step
Chatbot :  Hi there, how can I help?

Listening ......

what is your delivery time
You :  what is your delivery time
1/1 [==============================] - 0s 14ms/step
Chatbot :  Delivery takes 2-4 days

Listening ......

can I pay with cash
You :  can I pay with cash
1/1 [==============================] - 0s 15ms/step
Chatbot :  We accept most major credit cards, and Paypal

Listening ......

okay thank you
You :  okay thank you
1/1 [==============================] - 0s 14ms/step
Chatbot :  My pleasure

Listening ......

bye
You :  bye
1/1 [==============================] - 0s 17ms/step
Chatbot :  See you later, thanks for visiting
